# TAG 4 – Pandas: Tabellen-Denken & KPI-Logik

## Ziel dieses Notebooks

Nach diesem Notebook kannst du:

- Transaktionsdaten strukturiert analysieren
- KPIs mit groupby ableiten
- Datenquellen mit merge kombinieren
- Missing Values bewusst behandeln
- Interview-Fragen zu Pandas ruhig beantworten

Wichtig:
Wir lernen hier nicht „Pandas-Syntax“.
Wir lernen, wie man Geschäftslogik in Tabellen übersetzt.


In [69]:
import pandas as pd
import numpy as np

## Beispiel: Transaktionsdaten

Wir simulieren typische E-Commerce-Daten:

- customer_id
- country
- revenue
- order_date
- segment

Ziel: KPIs berechnen.


In [70]:
data = {
    "customer_id": [1, 2, 1, 3, 2, 4, 5, 3],
    "country": ["AT", "DE", "AT", "DE", "DE", "AT", "AT", "DE"],
    "revenue": [200, 50, 150, 300, 80, 400, 60, 120],
    "segment": ["Premium", "Standard", "Premium", "Premium", "Standard", "Premium", "Standard", "Premium"]
}

df = pd.DataFrame(data)
df

,customer_id,country,revenue,segment
0,1,AT,200,Premium
1,2,DE,50,Standard
2,1,AT,150,Premium
3,3,DE,300,Premium
4,2,DE,80,Standard
5,4,AT,400,Premium
6,5,AT,60,Standard
7,3,DE,120,Premium


## Was ist groupby?

Intuition:
Ich fasse Daten entlang einer fachlichen Dimension zusammen.

Beispiel:
„Wie viel Umsatz pro Land?“
„Wie viel Umsatz pro Kundensegment?“

Warum existiert es?
Weil Rohdaten keine Entscheidung ermöglichen.
Erst Aggregation erzeugt KPI.


In [71]:
df.groupby("country")["revenue"].sum()
df.groupby("segment")["revenue"].mean()


segment
Premium     234.000000
Standard     63.333333
Name: revenue, dtype: float64

Business-Frage:
Welches Segment ist wertvoller?


## Mehrere Kennzahlen gleichzeitig berechnen


In [72]:
df.groupby("country").agg({
    "revenue": ["sum", "mean", "count"]
})


revenue             
            sum   mean count
country                     
AT          810  202.5     4
DE          550  137.5     4

Jetzt sehen wir:

- Gesamtumsatz
- Durchschnittlicher Umsatz
- Anzahl der Bestellungen

Das ist echte KPI-Logik. KPI = Key PErformance Indicator


## Warum merge?

In Realität liegen Daten selten in einer Tabelle.

Beispiel:
- Transaktionen
- Kundensegmente
- Marketingdaten

merge verbindet logisch zusammengehörige Informationen.


In [73]:
customers = pd.DataFrame({
    "customer_id": [1,2,3,4,5],
    "loyalty_score": [80, 40, 90, 70, 50]
})

merged_df = df.merge(customers, on="customer_id")
merged_df


,customer_id,country,revenue,segment,loyalty_score
0,1,AT,200,Premium,80
1,2,DE,50,Standard,40
2,1,AT,150,Premium,80
3,3,DE,300,Premium,90
4,2,DE,80,Standard,40
5,4,AT,400,Premium,70
6,5,AT,60,Standard,50
7,3,DE,120,Premium,90


Jetzt können wir z. B. analysieren:

Haben Kunden mit hohem Loyalty-Score höheren Umsatz?


## Warum ist apply problematisch?

Intuition:
apply führt Python-Code zeilenweise aus.
Das ist langsamer als vektorisierte Operationen.

Regel:
Nutze zuerst native Pandas-Funktionen.


In [74]:
# NICHT optimal
df["revenue_category"] = df["revenue"].apply(lambda x: "High" if x > 150 else "Low")
df


,customer_id,country,revenue,segment,revenue_category
0,1,AT,200,Premium,High
1,2,DE,50,Standard,Low
2,1,AT,150,Premium,Low
3,3,DE,300,Premium,High
4,2,DE,80,Standard,Low
5,4,AT,400,Premium,High
6,5,AT,60,Standard,Low
7,3,DE,120,Premium,Low


In [75]:
df["revenue_category"] = np.where(df["revenue"] > 150, "High", "Low")
df


,customer_id,country,revenue,segment,revenue_category
0,1,AT,200,Premium,High
1,2,DE,50,Standard,Low
2,1,AT,150,Premium,Low
3,3,DE,300,Premium,High
4,2,DE,80,Standard,Low
5,4,AT,400,Premium,High
6,5,AT,60,Standard,Low
7,3,DE,120,Premium,Low


## Missing Values

Missing bedeutet nicht automatisch Fehler.

Fragen:
- Ist es wirklich 0?
- Ist es unbekannt?
- Ist es systematisch?

Blindes dropna() kann gefährlich sein.


In [76]:
df.loc[2, "revenue"] = np.nan
df


,customer_id,country,revenue,segment,revenue_category
0,1,AT,200.0,Premium,High
1,2,DE,50.0,Standard,Low
2,1,AT,NaN,Premium,Low
3,3,DE,300.0,Premium,High
4,2,DE,80.0,Standard,Low
5,4,AT,400.0,Premium,High
6,5,AT,60.0,Standard,Low
7,3,DE,120.0,Premium,Low


In [77]:
#Option 1: Zeile entfernen
df = df.dropna()

df


,customer_id,country,revenue,segment,revenue_category
0,1,AT,200.0,Premium,High
1,2,DE,50.0,Standard,Low
3,3,DE,300.0,Premium,High
4,2,DE,80.0,Standard,Low
5,4,AT,400.0,Premium,High
6,5,AT,60.0,Standard,Low
7,3,DE,120.0,Premium,Low


In [78]:
#Option 2: NaN mit einem Wert ersetzen
df = df["revenue"].fillna(0)

df

0    200.0
1     50.0
3    300.0
4     80.0
5    400.0
6     60.0
7    120.0
Name: revenue, dtype: float64

Business-Entscheidung:
Ist fehlender Umsatz = 0 oder unbekannt?


# Interview-Training – Pandas

Warum ist groupby wichtig?

„groupby ist wichtig, weil ich damit Transaktionsdaten entlang fachlicher Dimensionen wie Kunde oder Land zusammenfassen kann, um KPIs abzuleiten.“

Wann nutzt du merge?

„Wenn Informationen aus verschiedenen Datenquellen logisch zusammengehören.“

Warum ist apply problematisch?

„Weil es zeilenweise in Python läuft und langsamer ist als vektorisierte Operationen.“

Wie gehst du mit Missing Values um?

„Kontextabhängig – entweder entfernen oder sinnvoll ersetzen.“


## Mini-Projekt

1) Berechne Gesamtumsatz pro Kunde
2) Finde Top-2 Kunden nach Umsatz
3) Berechne Durchschnittsumsatz pro Segment
4) Erstelle eine Spalte: HighValue (Umsatz > 200)


In [79]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# Mini-Projekt Dataset: E-Commerce Transaktionen (synthetisch)
# Ziel: KPI-Analyse (Umsatz pro Kunde, Top-Kunden, Segment-Analyse, HighValue)
# ------------------------------------------------------------

data = {
    "customer_id": [1, 2, 1, 3, 2, 4, 5, 3],
    "country":     ["AT", "DE", "AT", "DE", "DE", "AT", "AT", "DE"],
    "revenue":     [200.0, 50.0, 150.0, "", 80.0, 400.0, 60.0, 120.0],
    "segment":     ["Premium", "Standard", "Premium", "Premium", "Standard", "Premium", "Standard", "Premium"],
    "order_date":  pd.to_datetime(["2025-01-05", "2025-01-06", "2025-01-09", "2025-01-10",
                                   "2025-01-11", "2025-01-12", "2025-01-14", "2025-01-15"])
}

df = pd.DataFrame(data)

# Optional: bewusst ein Missing Value einbauen, um Missing-Value-Handling zu üben
# (Kommentar entfernen, wenn du das im Mini-Projekt testen willst)
df.loc[2, "revenue"] = np.nan

# Basic sanity checks (hilft dir, nicht aus Versehen mit Series zu arbeiten)
print("Type:", type(df))
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

df


Type: <class 'pandas.DataFrame'>
Shape: (8, 5)
Columns: ['customer_id', 'country', 'revenue', 'segment', 'order_date']


,customer_id,country,revenue,segment,order_date
0,1,AT,200.0,Premium,2025-01-05
1,2,DE,50.0,Standard,2025-01-06
2,1,AT,NaN,Premium,2025-01-09
3,3,DE,,Premium,2025-01-10
4,2,DE,80.0,Standard,2025-01-11
5,4,AT,400.0,Premium,2025-01-12
6,5,AT,60.0,Standard,2025-01-14
7,3,DE,120.0,Premium,2025-01-15


1) Berechne Gesamtumsatz pro Kunde

In [80]:
clean_df = df.dropna()

clean_df["revenue"] = pd.to_numeric(clean_df["revenue"], errors="coerce") 

revenue_sum = clean_df["revenue"].sum()
print("Gesamtumsatz:", revenue_sum, "€")


Gesamtumsatz: 910.0 €


2) Finde Top-2 Kunden nach Umsatz

In [81]:
# Um die 2 Top Kunde nach Umsatz zu finden
top_customers = clean_df.groupby("customer_id")["revenue"].sum().nlargest(2) 
# Diese Zeile gruppiert die Daten nach "customer_id", 
# summiert den Umsatz für jeden Kunden und gibt die 2 Kunden mit dem höchsten Umsatz zurück.
print("Top Kunden nach Umsatz:")

for customer_id, revenue in top_customers.items():
    print(f"Kunde {customer_id}: {revenue:.2f} €")

Top Kunden nach Umsatz:
Kunde 4: 400.00 €
Kunde 1: 200.00 €


3) Berechne Durchschnittsumsatz pro Segment

In [82]:
mean_revenue_by_segment = clean_df.groupby("segment")["revenue"].mean()

print("Durchschnittlicher Umsatz pro Segment:\n")

for segment, value in mean_revenue_by_segment.items():
    print(f"{segment}: {value:.2f} €")


Durchschnittlicher Umsatz pro Segment:

Premium: 240.00 €
Standard: 63.33 €


4) Erstelle eine Spalte: HighValue (Umsatz > 200)